In [1]:
import cv2
import numpy as np

COL_BLUE = (255, 0, 0)
COL_GREEN = (0, 255, 0)
COL_RED = (0, 0, 255)
FONT = cv2.FONT_HERSHEY_SIMPLEX
FONT_SCALE = 1
THICKNESS = 1
LINE_TYPE = 2


def length(start, end):
    return np.sqrt((start[0] - end[0]) ** 2 + (start[1] - end[1]) ** 2)


def get_angle(start, rotation_center, end):
    sr = length(start, rotation_center)
    re = length(rotation_center, end)
    se = length(start, end)
    cos_angle = (se ** 2 - sr ** 2 - re ** 2) / (-2 * sr * re)
    return np.arccos(cos_angle) / np.pi * 180


def rotation(image, angle):
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return result


def find_marker_in_image(image, marker, allowed_rotations):
    # Returns location of maximum response with all allowed rotations
    best_val = -np.inf
    best_loc = None
    for rot in allowed_rotations:
        rotated_marker = rotation(marker, rot)
        matched = cv2.matchTemplate(image, rotated_marker, cv2.TM_CCORR_NORMED)
        _, max_val, _, top_left = cv2.minMaxLoc(matched)
        if max_val > best_val:
            best_val = max_val
            best_loc = top_left
    return best_loc


def get_marker_center(marker, center_offset):
    if center_offset is None:
        marker_h, marker_w = marker.shape[:-1]
        return (marker_w // 2, marker_h // 2)
    return center_offset


def show_marker(marker):
    marker_img, _, _, marker_center = marker
    marker_center = get_marker_center(marker_img, marker_center)
    copied_img = marker_img.copy()
    cv2.circle(copied_img, marker_center, 5, COL_BLUE, -1)
    cv2.imshow('frame', copied_img)
    cv2.waitKey(0)


def pick_marker(marker_img, marker_name: str):
    y_offset = 0

    img = marker_img.copy()

    def on_trackbar(val):
        nonlocal y_offset
        y_offset = val
        cv2.imshow('image', img[y_offset:, :, :])

    cv2.putText(img, f"Pick top left of marker {marker_name}", (30, 80), FONT, FONT_SCALE, COL_BLUE, THICKNESS, LINE_TYPE)
    on_trackbar(y_offset)
    cv2.createTrackbar("Y - Offset", 'image', 0, img.shape[0], on_trackbar)

    top_left_coords, bottom_right_coords, center_coords = None, None, None

    def click_event(event, x, y, flags, params):
        nonlocal top_left_coords, bottom_right_coords, center_coords, img, y_offset
        if event == cv2.EVENT_LBUTTONDOWN:

            if top_left_coords is None:
                top_left_coords = (x, y + y_offset)
                img = marker_img.copy()
                cv2.circle(img, top_left_coords, 5, COL_BLUE, -1)
                cv2.putText(img, f"Pick bottom right of marker {marker_name}", (30, 80), FONT, FONT_SCALE, COL_BLUE, THICKNESS, LINE_TYPE)
                on_trackbar(y_offset)
            elif bottom_right_coords is None:
                bottom_right_coords = (x, y + y_offset)
                img = marker_img.copy()
                cv2.circle(img, top_left_coords, 5, COL_BLUE, -1)
                cv2.circle(img, bottom_right_coords, 5, COL_BLUE, -1)
                cv2.putText(img, f"Pick center of marker {marker_name} (Optional). Press any button to continue.", (30, 80), FONT,
                            FONT_SCALE, COL_BLUE, THICKNESS, LINE_TYPE)
                on_trackbar(y_offset)
            else:
                center_coords = (x, y + y_offset)
                cv2.destroyAllWindows()

    cv2.setMouseCallback('image', click_event)
    # wait for a key to be pressed to exit
    cv2.waitKey(0)

    # close the window
    cv2.destroyAllWindows()
    print(top_left_coords, bottom_right_coords, center_coords)
    return (marker_img[top_left_coords[1]:bottom_right_coords[1], top_left_coords[0]:bottom_right_coords[0], :], top_left_coords,
            (center_coords[0] - top_left_coords[0], center_coords[1] - top_left_coords[1]))

In [3]:
marker_img_1 = cv2.imread("frame0.jpg", cv2.IMREAD_COLOR)
trochanter_idx, knee_idx, ankle_idx, foot_marker_1_idx, foot_marker_2_idx, shoulder_idx, elbow_idx, wrist_idx = 0, 1, 2, 3, 4, 5, 6, 7
trochanter, trochanter_initial_coords, trochanter_center = pick_marker(marker_img_1, "Trochanter")
knee, knee_initial_coords, knee_center = pick_marker(marker_img_1, "Knee")
ankle, ankle_initial_coords, ankle_center = pick_marker(marker_img_1, "Ankle")
foot_marker_1, foot_marker_1_initial_coords, foot_marker_1_center = pick_marker(marker_img_1, "Foot marker 1")
foot_marker_2, foot_marker_2_initial_coords, foot_marker_2_center = pick_marker(marker_img_1, "Foot marker 2")
shoulder, shoulder_initial_coords, shoulder_center = pick_marker(marker_img_1, "Shoulder")
elbow, elbow_initial_coords, elbow_center = pick_marker(marker_img_1, "Elbow")
wrist, wrist_initial_coords, wrist_center = pick_marker(marker_img_1, "Wrist")

markers = [(trochanter, [0], trochanter_initial_coords, trochanter_center), (knee, [0], knee_initial_coords, knee_center),
           (ankle, [0], ankle_initial_coords, ankle_center),
           (foot_marker_1, [-30, -15, 0, 15, 30], foot_marker_1_initial_coords, foot_marker_1_center),
           (foot_marker_2, [-30, -15, 0, 15, 30], foot_marker_2_initial_coords, foot_marker_2_center),
           (shoulder, [0], shoulder_initial_coords, shoulder_center),
           (elbow, [0], elbow_initial_coords, elbow_center),
           (wrist, [0], wrist_initial_coords, wrist_center)]
show_marker(markers[trochanter_idx])

(86, 541) (130, 583) (105, 562)
(270, 836) (286, 857) (280, 848)
(118, 1092) (137, 1114) (128, 1102)
(85, 1127) (119, 1165) (99, 1148)
(178, 1181) (203, 1208) (191, 1198)
(437, 335) (461, 362) (449, 350)
(487, 582) (515, 609) (500, 597)
(653, 601) (684, 617) (668, 610)


In [4]:
EXPORT_UNEDITED_FRAMES = [0]
EXPORT_EDITED_FRAMES = [0]
CROP_START_FRAMECOUNT = 0  # Set this to when you want to start the video
CROP_END_FRAMECOUNT = 491  # Set this to when you want to end the video
ROTATION = True  # True if rotated to the right by 90deg
ROM_PIXELS = 50  # number of maximal pixels of movement between frames

KNEE_ANGLE_COLOR = COL_RED
ANKLE_ANGLE_COLOR = (168, 50, 117)
ANKLE_GROUND_ANGLE_COLOR = (252, 90, 3)
HIP_ANGLE_COLOR = COL_GREEN
ELBOW_ANGLE_COLOR = KNEE_ANGLE_COLOR

FRAME_COUNT_TEXT = (30, 50)
KNEE_ANGLE_TEXT = (30, 80)
ANKLE_ANGLE_TEXT = (30, 110)
ANKLE_GROUND_ANGLE_TEXT = (30, 140)
HIP_ANGLE_TEXT = (30, 170)
ELBOW_ANGLE_TEXT = (240, 50)

VID_NAME = 'Saddle-5mmWithTiltNewBarsAero.MOV'

vid = cv2.VideoCapture(VID_NAME)
height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
fps = vid.get(cv2.CAP_PROP_FPS)
framecount = vid.get(cv2.CAP_PROP_FRAME_COUNT)
print(f"height:{height} width:{width}, fps: {fps}, framecount: {framecount}")

out = cv2.VideoWriter('output.avi', cv2.VideoWriter_fourcc(*"MJPG"), fps, (height, width))
count = 0
marker_coords = []
middle_points = []
# Read until video is completed
while vid.isOpened():
    # Capture frame-by-frame
    ret, frame = vid.read()
    if not ret:
        break
    if CROP_START_FRAMECOUNT <= count <= CROP_END_FRAMECOUNT:
        frame = cv2.rotate(frame, cv2.ROTATE_90_COUNTERCLOCKWISE)
        cv2.putText(frame, f"Frame: {count}", FRAME_COUNT_TEXT, FONT, FONT_SCALE, COL_BLUE, THICKNESS, LINE_TYPE)
        if count in EXPORT_UNEDITED_FRAMES:
            cv2.imwrite(f"frame{count}.jpg", frame)
        if count == CROP_START_FRAMECOUNT:
            for marker, rotations, initial_coords, center_offset in markers:
                top_left = initial_coords
                marker_h, marker_w = marker.shape[:-1]
                marker_center = get_marker_center(marker, center_offset)
                middle_point = (top_left[0] + marker_center[0], top_left[1] + marker_center[1])
                bottom_right = (top_left[0] + marker_w, top_left[1] + marker_h)
                cv2.rectangle(frame, top_left, bottom_right, COL_BLUE, 2)
                cv2.circle(frame, middle_point, 5, COL_BLUE, -1)
                marker_coords.append([top_left])
                middle_points.append([middle_point])
        else:
            for (idx, (marker, rotations, _, center_offset)) in enumerate(markers):
                last_coords = marker_coords[idx][-1]
                window_left = min(ROM_PIXELS, last_coords[0])
                window_right = min(ROM_PIXELS, frame.shape[1] - last_coords[0])
                window_top = min(ROM_PIXELS, last_coords[1])
                window_bottom = min(ROM_PIXELS, frame.shape[0] - last_coords[1])
                marker_h, marker_w = marker.shape[:-1]


                def transform_coordinates(hit_coordinates):
                    return hit_coordinates[0] + last_coords[0] - window_left, hit_coordinates[1] + last_coords[1] - window_top


                max_loc = find_marker_in_image(frame[last_coords[1] - window_top: last_coords[1] + marker_h + window_bottom,
                                               last_coords[0] - window_left: last_coords[0] + marker_w + window_right, :],
                                               marker, rotations)
                top_left = transform_coordinates(max_loc)
                marker_center = get_marker_center(marker, center_offset)
                middle_point = (top_left[0] + marker_center[0], top_left[1] + marker_center[1])
                bottom_right = (top_left[0] + marker_w, top_left[1] + marker_h)
                cv2.rectangle(frame, top_left, bottom_right, COL_BLUE, 2)
                cv2.circle(frame, middle_point, 5, COL_BLUE, -1)
                marker_coords[idx].append(top_left)
                middle_points[idx].append(middle_point)
                if idx == 1:
                    print(count, abs(last_coords[0] - top_left[0]) + abs(last_coords[1] - top_left[1]))
                    #cv2.imshow("Frame", matched)
                    #cv2.waitKey(1)
        # Write knee angle into the video:
        trochanter_pt = middle_points[trochanter_idx][-1]
        knee_pt = middle_points[knee_idx][-1]
        ankle_pt = middle_points[ankle_idx][-1]
        knee_angle = get_angle(trochanter_pt, knee_pt, ankle_pt)
        cv2.line(frame, trochanter_pt, knee_pt, KNEE_ANGLE_COLOR, 2)
        cv2.line(frame, knee_pt, ankle_pt, KNEE_ANGLE_COLOR, 2)
        cv2.putText(frame, f"Knee angle: {round(knee_angle, 2)}", KNEE_ANGLE_TEXT, FONT, FONT_SCALE, KNEE_ANGLE_COLOR, THICKNESS, LINE_TYPE)

        # Write ankle angles into the video:
        knee_pt = middle_points[knee_idx][-1]
        foot_1_pt = middle_points[foot_marker_1_idx][-1]
        foot_2_pt = middle_points[foot_marker_2_idx][-1]
        foot_2_projection = (foot_2_pt[0], foot_1_pt[1])
        ankle_angle = get_angle(foot_2_pt, foot_1_pt, knee_pt)

        cv2.line(frame, knee_pt, foot_1_pt, ANKLE_ANGLE_COLOR, 2)
        cv2.line(frame, foot_1_pt, foot_2_pt, ANKLE_ANGLE_COLOR, 2)
        cv2.putText(frame, f"Ankle angle: {round(ankle_angle, 2)}", ANKLE_ANGLE_TEXT, FONT, FONT_SCALE, ANKLE_ANGLE_COLOR, THICKNESS,
                    LINE_TYPE)

        ankle_ground_angle = -get_angle(foot_2_pt, foot_1_pt, foot_2_projection) if foot_1_pt[1] <= foot_2_pt[1] else get_angle(
            foot_2_projection, foot_1_pt, foot_2_pt)
        cv2.line(frame, (foot_1_pt[0] - 50, foot_1_pt[1]), (foot_2_pt[0] + 50, foot_1_pt[1]), ANKLE_GROUND_ANGLE_COLOR, 2)
        cv2.putText(frame, f"Ankle ground angle: {round(ankle_ground_angle, 2)}", ANKLE_GROUND_ANGLE_TEXT, FONT, FONT_SCALE,
                    ANKLE_GROUND_ANGLE_COLOR, THICKNESS, LINE_TYPE)

        # Write hip angle into the video
        knee_pt = middle_points[knee_idx][-1]
        trochanter_pt = middle_points[trochanter_idx][-1]
        shoulder_pt = middle_points[shoulder_idx][-1]
        hip_angle = get_angle(knee_pt, trochanter_pt, shoulder_pt)
        cv2.line(frame, knee_pt, trochanter_pt, HIP_ANGLE_COLOR, 2)
        cv2.line(frame, trochanter_pt, shoulder_pt, HIP_ANGLE_COLOR, 2)
        cv2.putText(frame, f"Hip angle: {round(hip_angle, 2)}", HIP_ANGLE_TEXT, FONT, FONT_SCALE, HIP_ANGLE_COLOR, THICKNESS,
                    LINE_TYPE)
        
        # Write elbow angle into the video
        wrist_pt = middle_points[wrist_idx][-1]
        elbow_pt = middle_points[elbow_idx][-1]
        shoulder_pt = middle_points[shoulder_idx][-1]
        elbow_angle = get_angle(wrist_pt, elbow_pt, shoulder_pt)
        cv2.line(frame, wrist_pt, elbow_pt, ELBOW_ANGLE_COLOR, 2)
        cv2.line(frame, elbow_pt, shoulder_pt, ELBOW_ANGLE_COLOR, 2)
        cv2.putText(frame, f"Elbow angle: {round(elbow_angle, 2)}", ELBOW_ANGLE_TEXT, FONT, FONT_SCALE, ELBOW_ANGLE_COLOR, THICKNESS,
                    LINE_TYPE)
        
        if count in EXPORT_EDITED_FRAMES:
            cv2.imwrite(f"frame{count}marked.jpg", frame)
        out.write(frame)
        if count % 50 == 0:
            print(count)
    count += 1

# When everything done, release the video capture object
vid.release()
out.release()
# Closes all the frames
cv2.destroyAllWindows()

height:720 width:1280, fps: 50.0, framecount: 521.0
0
1 16
2 21
3 21
4 25
5 28
6 28
7 30
8 32
9 29
10 28
11 25
12 23
13 18
14 15
15 7
16 1
17 7
18 13
19 18
20 22
21 25
22 30
23 33
24 37
25 37
26 34
27 33
28 30
29 24
30 19
31 12
32 6
33 1
34 6
35 12
36 15
37 18
38 20
39 22
40 25
41 27
42 30
43 32
44 33
45 32
46 31
47 26
48 22
49 16
50 10
50
51 4
52 2
53 9
54 15
55 20
56 24
57 27
58 30
59 33
60 36
61 36
62 34
63 32
64 27
65 22
66 16
67 11
68 4
69 4
70 8
71 12
72 14
73 19
74 22
75 24
76 27
77 28
78 31
79 35
80 33
81 32
82 28
83 24
84 19
85 13
86 9
87 2
88 5
89 10
90 17
91 20
92 27
93 30
94 34
95 35
96 36
97 36
98 36
99 30
100 24
100
101 20
102 14
103 8
104 1
105 6
106 9
107 14
108 18
109 21
110 23
111 26
112 27
113 31
114 32
115 32
116 31
117 30
118 28
119 22
120 16
121 12
122 4
123 3
124 7
125 16
126 20
127 23
128 28
129 32
130 33
131 38
132 36
133 36
134 31
135 27
136 22
137 15
138 10
139 3
140 3
141 7
142 12
143 16
144 20
145 21
146 25
147 26
148 29
149 31
150 32
150
151 34
152 31
153 